## Model tests

In [1]:
import sys
import torch
import torch.nn as nn

sys.path.append("../source/")
from model import Model

In [2]:
sizes = [2, 1, 2, 3]
activations = 3 * [nn.ReLU()] + [None]

In [3]:
mlp = Model(sizes, activations, device=torch.device("mps"))
print("Number of parameters:", mlp.num_parameters())

Number of parameters: 16


In [4]:
sampled_params = mlp.get_parameters()
print("Initial parameters:", sampled_params)

Initial parameters: tensor([ 0.2802, -0.0793,  0.3519, -0.4749,  0.9992,  0.2728,  0.4385,  0.5089,
         0.3253, -0.1857, -0.4612, -0.5385, -0.6866,  0.2795,  0.1864,  0.4733],
       device='mps:0', grad_fn=<CatBackward0>)


In [5]:
updated_params = sampled_params + torch.rand(len(sampled_params)).to(sampled_params.device)
print("New parameters:", updated_params)

New parameters: tensor([ 0.9172,  0.0430,  1.0029,  0.2325,  1.0572,  0.3611,  0.7081,  1.1074,
         1.2253,  0.6409, -0.4033,  0.1014,  0.0534,  1.2560,  0.7285,  1.2201],
       device='mps:0', grad_fn=<AddBackward0>)


In [6]:
mlp.set_parameters(updated_params)
updated_sampled_params = mlp.get_parameters()
print("Updated parameters:", updated_sampled_params)

Updated parameters: tensor([ 0.9172,  0.0430,  1.0029,  0.2325,  1.0572,  0.3611,  0.7081,  1.1074,
         1.2253,  0.6409, -0.4033,  0.1014,  0.0534,  1.2560,  0.7285,  1.2201],
       device='mps:0', grad_fn=<CatBackward0>)


In [7]:
#grad = mlp.get_grad()
#print("Initial grad:", grad)

#updated_grad = grad + torch.rand(len(sampled_params))
#print("New grad:", updated_grad)

#mlp.set_grad(updated_grad)
#updated_grad = mlp.get_grad()
#print("Updated grad:", updated_grad)

In [8]:
mlp

Model(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=1, bias=True)
    (1): Linear(in_features=1, out_features=2, bias=True)
    (2): Linear(in_features=2, out_features=3, bias=True)
  )
  (loss): CrossEntropyLoss()
)

In [9]:
batch_size = 10
x = torch.rand(batch_size, sizes[0])
y = torch.randint(low=0, high=sizes[-1], size=(batch_size, ))
x, y

(tensor([[0.2892, 0.7407],
         [0.5225, 0.3603],
         [0.4359, 0.8282],
         [0.7688, 0.9673],
         [0.1713, 0.1388],
         [0.8014, 0.1350],
         [0.3062, 0.5511],
         [0.6520, 0.7147],
         [0.7551, 0.9675],
         [0.3891, 0.5407]]),
 tensor([2, 0, 0, 0, 2, 2, 2, 2, 1, 2]))

In [10]:
mlp.loss(mlp(x), y.to(mlp.device))

tensor(25.4043, device='mps:0', grad_fn=<NllLossBackward0>)

In [11]:
mlp.compute_log_likelihood(x, y)

tensor(-25.4043, device='mps:0', grad_fn=<MulBackward0>)

In [12]:
mlp.compute_log_prior()

tensor(-19.9848, device='mps:0', grad_fn=<MulBackward0>)

In [13]:
log_target = mlp.compute_log_target(x, y)

In [14]:
mlp.compute_grad_log_target(log_target)

tensor([ -5.8462,  -5.5962, -11.0059,  -9.0902, -13.1004,  -6.4912,  -8.8217,
         -5.7267, -16.0900,   0.0522,   2.7179,   3.8249,  12.4967,  -7.8302,
          0.1677,   4.4578], device='mps:0', grad_fn=<CatBackward0>)